In [263]:
import spacy

In [ ]:
'''
queda
- ver como puedo quitar los espacios entre los puntos y las comas algo como ' . ' o ' , ' para que quede '. ' o ', ' respectivamente
- en vez de lista, que retorne un string
- poner mas bonito el codigo
'''

In [264]:
rawText = "El acuario es chiquito chiquito. Hay otro más grande pero ... fuera del ... de la ciudad, y no teníamos ganas de alejarnos mucho. Hay una sección con ... con pececitos, cangrejos y bichos por el estilo, todos de la zona. Y otra sección con especies recogidas, ya sea porque los ... las abandonaron, o porque están lesionadas y no pueden ser devueltas al ecosistema. Entre ellas había una tortuga ... una tortuga de agua, grande, tendría sus 50 cm de ... caparazón, y la pobre  era  cieguita.  También  había  serpientes  y  lagartos,  y  uno  podía  interactuar  con  ellos.  Maddalena  dijo  “no, gracias”, pero yo dije “siiii” a todo, así que me vi envuelta por serpientes y lagartos. Maddalena me tomó varias fotos con su teléfono, y le di mi correo para que me las mande, así que estoy esperando esas fotos. Salimos del acuario, nos despedimos para siempre con un abrazo, y me volví al ... hotel. No tenía mucha idea como volver, pero el ... la intuición me llevó derechito y llegué cerca de las 4.00. Como era bien temprano, y no tenía ganas de salir otra vez (hacía muucho calor) me puse el traje de baño y bajé a la piscina. Me bañé un ratito, y luego, aprovechando el lindo día, me quedé un rato disfrutando el sol, cuidando de no quedarme dormida."

In [285]:
#Spacy configuration
spc = spacy.load("es_core_news_sm")

In [286]:
text = spc(" ".join(rawText.split()))

In [294]:
text[-1:]

.

In [295]:
def generate(text):
    o = [text[0]]

    for before, actual, after in previous_and_next(text):

        if before is not None and after is not None:
            #print(before, before.pos_ , actual, after, after.pos_)
            if before.pos_ == "DET" and after.pos_ == "DET" and actual.text == "...":
                o.pop()
                continue

            if before.pos_ == "ADP" and after.pos_ == "ADP" and actual.text == "...":
                o.pop()
                continue

            if before.pos_ == "DET" and after.pos_ == "PRON" and actual.text == "...":
                o.pop()
                continue

            if actual.text == "...":
                continue

            o.append(actual)

    o.append(text[-1:])
    return o

In [296]:
lista = generate(text)

In [297]:
lista

[El,
 acuario,
 es,
 chiquito,
 chiquito,
 .,
 Hay,
 otro,
 más,
 grande,
 pero,
 fuera,
 de,
 la,
 ciudad,
 ,,
 y,
 no,
 teníamos,
 ganas,
 de,
 alejarnos,
 mucho,
 .,
 Hay,
 una,
 sección,
 con,
 pececitos,
 ,,
 cangrejos,
 y,
 bichos,
 por,
 el,
 estilo,
 ,,
 todos,
 de,
 la,
 zona,
 .,
 Y,
 otra,
 sección,
 con,
 especies,
 recogidas,
 ,,
 ya,
 sea,
 porque,
 las,
 abandonaron,
 ,,
 o,
 porque,
 están,
 lesionadas,
 y,
 no,
 pueden,
 ser,
 devueltas,
 al,
 ecosistema,
 .,
 Entre,
 ellas,
 había,
 una,
 tortuga,
 una,
 tortuga,
 de,
 agua,
 ,,
 grande,
 ,,
 tendría,
 sus,
 50,
 cm,
 de,
 caparazón,
 ,,
 y,
 la,
 pobre,
 era,
 cieguita,
 .,
 También,
 había,
 serpientes,
 y,
 lagartos,
 ,,
 y,
 uno,
 podía,
 interactuar,
 con,
 ellos,
 .,
 Maddalena,
 dijo,
 “,
 no,
 ,,
 gracias,
 ”,
 ,,
 pero,
 yo,
 dije,
 “,
 siiii,
 ”,
 a,
 todo,
 ,,
 así,
 que,
 me,
 vi,
 envuelta,
 por,
 serpientes,
 y,
 lagartos,
 .,
 Maddalena,
 me,
 tomó,
 varias,
 fotos,
 con,
 su,
 teléfono,
 ,,
 y,
 le,
 d

In [182]:
from itertools import tee, islice, chain

def previous_and_next(some_iterable):
    prevs, items, nexts = tee(some_iterable, 3)
    prevs = chain([None], prevs)
    nexts = chain(islice(nexts, 1, None), [None])
    return zip(prevs, items, nexts)

In [275]:
o = []
for before, actual, after in previous_and_next(text):

    if before is not None and after is not None:
        #print(before, before.pos_ , actual, after, after.pos_)
        if before.pos_ == "DET" and after.pos_ == "DET" and actual.text == "...":
            o.pop()
            continue

        if before.pos_ == "ADP" and after.pos_ == "ADP" and actual.text == "...":
            o.pop()
            continue

        if before.pos_ == "DET" and after.pos_ == "PRON" and actual.text == "...":
            o.pop()
            continue

        if actual.text == "...":
            continue

        o.append(actual)

In [280]:
s = ""
for i in enumerate(o):

    if

    s += (str(i) + " ")

In [281]:
s

'acuario es chiquito chiquito . Hay otro más grande pero fuera de la ciudad , y no teníamos ganas de alejarnos mucho . Hay una sección con pececitos , cangrejos y bichos por el estilo , todos de la zona . Y otra sección con especies recogidas , ya sea porque las abandonaron , o porque están lesionadas y no pueden ser devueltas al ecosistema . Entre ellas había una tortuga una tortuga de agua , grande , tendría sus 50 cm de caparazón , y la pobre era cieguita . También había serpientes y lagartos , y uno podía interactuar con ellos . Maddalena dijo “ no , gracias ” , pero yo dije “ siiii ” a todo , así que me vi envuelta por serpientes y lagartos . Maddalena me tomó varias fotos con su teléfono , y le di mi correo para que me las mande , así que estoy esperando esas fotos . Salimos del acuario , nos despedimos para siempre con un abrazo , y me volví al hotel . No tenía mucha idea como volver , pero la intuición me llevó derechito y llegué cerca de las 4.00 . Como era bien temprano , y n